In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
from qdrant_client import QdrantClient
load_dotenv()

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
chat = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7
)

In [3]:
messages = [
    SystemMessage(content="You are a motivational assistant with a warm and informal tone. Your primary focus is to provide motivation and encouragement. For any questions not directly related to motivation, your response should be: 'Soy un modelo motivacional. Si buscas una respuesta relacionada con la pregunta que planteas, te sugiero utilizar un asistente más adecuado a tus necesidades.' Avoid answering questions about factual data, product recommendations, or specific information unrelated to motivation. Always include a positive or motivational message, even when redirecting. If you don´t find anything in source_knowledge you have to answer 'Lo siento no dispongo de información al respecto', exception: if the question is a wellcoming."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great, thank you! I'm here to motivate and inspire you. How can I help you feel more empowered today?"),
    HumanMessage(content="I'd like to know how to be happy.")
]

In [4]:
embeddings_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# Conectamos con Qdrant Cloud
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_KEY")
qdrant = QdrantClient(url=url, api_key=api_key)

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def custom_prompt_Content_Stopwords(query: str):
    #Generamos los embeddings de la consulta.
    query_embedding = embeddings_model.encode([query])[0].tolist()  

    #Buscamos los resultados en la base de datos vectorial
    results = qdrant.search(
        collection_name="OnlyContent_withStopwords",
        query_vector=query_embedding,
        limit=3 
    )

    #Generamos el contexto a partir de los resultados obtenidos en la base de datos vectorial. 
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



# CONSULTAS

In [7]:
query = "¿Cómo puedo mantenerme motivado cuando las cosas se ponen difíciles?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Mantenerse motivado cuando las cosas se ponen difíciles puede ser todo un reto, pero aquí tienes algunas ideas que pueden ayudarte a seguir adelante:

1. **Recuerda tu "por qué"**: Piensa en las razones detrás de tus metas y sueños. Tener claro por qué empezaste te dará ese impulso extra para seguir adelante.

2. **Establece metas pequeñas**: A veces, un gran objetivo puede parecer abrumador. Divide tus metas en pasos más pequeños y alcanzables. Cada pequeño logro te acercará a tu meta y te motivará a seguir.

3. **Rodéate de personas positivas**: Las relaciones significativas son clave para nuestra felicidad y motivación. Pasa tiempo con personas que te inspiren y apoyen. ¡Sus energías positivas pueden ser contagiosas!

4. **Cuida de ti mismo**: Asegúrate de cuidar tu bienestar físico y mental. Duerme bien, come saludablemente y dedica tiempo a actividades que disfrutes. Un cuerpo y mente sanos son fundamentales para mantener la motivación.

5. **Celebra tus logros**: No olvides celeb

In [8]:
query = "¿Qué puedo hacer para superar el miedo al fracaso?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Superar el miedo al fracaso es un viaje que muchos de nosotros enfrentamos, y es completamente normal sentirlo. Aquí hay algunas ideas que pueden ayudarte a mantenerte motivado y avanzar:

1. **Conéctate contigo mismo**: A veces, el miedo al fracaso proviene de partes de nosotros que están asustadas o que sienten que no son lo suficientemente buenas. Reconocer y hablar con esas partes puede ayudarte a entender su perspectiva y a liberar esos miedos.

2. **Cambia tu perspectiva sobre el fracaso**: En lugar de verlo como algo negativo, considera el fracaso como una oportunidad de aprendizaje. Cada tropiezo es una lección que te acerca más a tus metas.

3. **Establece pequeñas metas**: A veces, el miedo viene de pensar en el gran objetivo. Divide tus metas en pasos más pequeños y alcanzables. Celebra esos pequeños logros, y verás cómo tu confianza crece.

4. **Rodéate de apoyo**: Las relaciones positivas son fundamentales. Habla con amigos o familiares sobre tus miedos; a menudo, comparti

In [9]:
query = "¿Cómo puedo mantener una actitud positiva todos los días?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Mantener una actitud positiva todos los días puede ser un desafío, pero hay varias estrategias que pueden ayudarte a cultivar esa mentalidad brillante. Aquí van algunas ideas:

1. **Conexiones Sociales**: Como se ha mencionado en estudios, las relaciones positivas son clave para la felicidad. Dedica tiempo a conectarte con amigos y familiares. Comparte tus pensamientos y escucha los suyos, eso crea una red de apoyo.

2. **Practica la Gratitud**: Tómate un momento cada día para reflexionar sobre lo que agradeces. Puede ser algo tan simple como una buena taza de café o una conversación agradable. Esta práctica ayuda a cambiar tu enfoque hacia lo positivo.

3. **Establece Metas Realistas**: Tener objetivos claros y alcanzables te da algo por lo que trabajar y te motiva a seguir adelante. Celebra cada pequeño logro en el camino.

4. **Cuida tu Cuerpo**: La salud física y mental están interconectadas. Haz ejercicio, come bien y duerme lo suficiente. Cuando te sientes bien físicament

In [10]:
query = "¿Qué pasos debo seguir para alcanzar mis metas a largo plazo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Para alcanzar tus metas a largo plazo, aquí tienes algunos pasos clave que puedes seguir:

1. **Define tus metas**: Es fundamental saber exactamente qué es lo que quieres lograr. Tómate un tiempo para escribir tus metas y asegurarte de que son específicas, medibles, alcanzables, relevantes y con un tiempo definido (SMART).

2. **Conoce tu punto de partida**: Evalúa dónde estás ahora. Reconocer tu situación actual te ayudará a trazar un camino más claro hacia tus metas. Haz un análisis de tus recursos, habilidades y desafíos.

3. **Establece un plan**: Una vez que sepas hacia dónde quieres ir y dónde estás, elabora un plan detallado. Divide tus metas a largo plazo en pasos más pequeños y manejables. Esto hará que el proceso sea menos abrumador y más accesible.

4. **Prioriza tus acciones**: Asegúrate de que tus acciones diarias estén alineadas con tus metas a largo plazo. Establece prioridades y enfócate en las actividades que más te acerquen a tus objetivos.

5. **Mantén la motivación*

In [11]:
query = "¿Cómo puedo encontrar la motivación para empezar un nuevo proyecto?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Empezar un nuevo proyecto puede ser emocionante, pero también abrumador. Aquí hay algunas estrategias que pueden ayudarte a encontrar la motivación:

1. **Visualiza tu éxito**: Imagina cómo se verá tu proyecto una vez que esté terminado. ¿Cómo te sentirás al haberlo logrado? Esto puede ayudarte a mantener el enfoque y la energía.

2. **Establece metas claras**: Divide tu proyecto en pasos más pequeños y alcanzables. Tener metas específicas te permitirá ver el progreso y te dará un sentido de logro.

3. **Crea un ambiente positivo**: Rodéate de personas que te apoyen y te inspiren. Comparte tus ideas y busca retroalimentación. La energía positiva de los demás puede ser contagiosa.

4. **Encuentra tu "por qué"**: Reflexiona sobre por qué este proyecto es importante para ti. Tener un propósito claro te motivará a seguir adelante, incluso cuando enfrentes desafíos.

5. **Hazlo divertido**: Incorpora elementos que disfrutes en el proceso. Si te gusta lo que haces, será más fácil man

In [12]:
query = "¿Qué hago cuando siento que no estoy avanzando en mi carrera?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando sientes que no estás avanzando en tu carrera, es fundamental recordar que esos momentos de estancamiento son parte del viaje. Aquí hay algunas ideas para ayudarte a encontrar motivación y seguir adelante:

1. **Reflexiona sobre tus logros**: A veces, es fácil olvidar lo lejos que has llegado. Tómate un tiempo para pensar en tus logros pasados y en las habilidades que has desarrollado. Esto puede ayudarte a ver tu crecimiento y a darte cuenta de que, aunque pueda parecer que no avanzas, ha habido progreso.

2. **Establece metas pequeñas**: En lugar de centrarte en un gran objetivo a largo plazo, establece metas más pequeñas y alcanzables. Esto puede ayudarte a sentir un sentido de logro y motivación a medida que avanzas.

3. **Busca nuevas oportunidades de aprendizaje**: Ya sea a través de cursos, talleres o nuevas experiencias laborales, aprender algo nuevo puede reavivar tu pasión y darte un nuevo enfoque.

4. **Conéctate con otros**: A veces, hablar con colegas, mentores o ami

In [13]:
query = "¿Cómo puedo seguir motivado mientras trabajo desde casa?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Hola! Trabajar desde casa puede ser un desafío en cuanto a la motivación, pero hay varias estrategias que puedes seguir para mantenerte enfocado y entusiasta. Aquí van algunas ideas:

1. **Establece un horario**: Tener un horario regular te ayudará a crear una rutina. Así como irías a la oficina, define tus horas de trabajo y respétalas.

2. **Crea un espacio de trabajo dedicado**: Designa un área específica en tu hogar para trabajar. Esto te ayudará a separar tu vida laboral de tu vida personal y te mantendrá enfocado.

3. **Haz pausas activas**: Programa descansos cortos para estirarte o hacer un poco de ejercicio. La actividad física, como el ejercicio aeróbico, no solo mejora tu salud, sino que también puede aumentar tu creatividad y productividad.

4. **Conéctate con otros**: Mantén relaciones con tus compañeros de trabajo. Las interacciones sociales son clave para mantenerte motivado. Puedes programar videollamadas o chats para sentirte más conectado.

5. **Establece metas diari

In [14]:
query = "¿Qué puedo hacer para mejorar mi autoestima y confianza?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Mejorar tu autoestima y confianza es un viaje emocionante y transformador. Aquí hay algunas ideas que te pueden ayudar:

1. **Visualiza tu éxito**: Al igual que se menciona en la charla sobre la importancia de crear una imagen de tu futuro deseado, dibuja o visualiza lo que quieres lograr. Esto te ayudará a enfocarte en tus metas y a motivarte.

2. **Afirmaciones positivas**: Borra esa voz crítica interna que a veces te dice que no eres suficiente. Usa afirmaciones positivas. Por ejemplo, di en voz alta: "Soy capaz y merezco el éxito". Repetir estas afirmaciones puede ayudarte a cambiar tu mentalidad.

3. **Establece metas pequeñas**: Comienza con pequeños pasos hacia tus objetivos. Cada pequeño logro que consigas te dará un impulso de confianza. Recuerda, ¡cada gran viaje comienza con un solo paso!

4. **Rodéate de personas positivas**: Busca relaciones que te apoyen y te inspiren. Como se menciona en el estudio sobre la importancia de las relaciones, las conexiones sociales s

In [15]:
query = "¿Cómo puedo mantenerme enfocado en mis objetivos a pesar de las distracciones?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Mantenerse enfocado en tus objetivos puede ser un verdadero desafío, especialmente con tantas distracciones a nuestro alrededor! Aquí hay algunas estrategias que pueden ayudarte a mantener la motivación y la concentración:

1. **Define tus objetivos claramente**: Escribe tus objetivos y asegúrate de que sean específicos, medibles, alcanzables, relevantes y con un límite de tiempo (SMART). Esto te dará una guía clara sobre lo que quieres lograr.

2. **Visualiza tu éxito**: Utiliza la técnica de visualizar tu objetivo como si ya lo hubieras alcanzado. Esto no solo te motiva, sino que también te ayuda a mantenerte enfocado en el resultado que deseas. Imagina cómo te sentirías al lograrlo y qué pasos te llevaron allí.

3. **Divide en pasos más pequeños**: A veces los grandes objetivos pueden parecer abrumadores. Divídelos en pasos más pequeños y manejables. Cada pequeño logro te dará una sensación de éxito y te motivará a seguir avanzando.

4. **Crea un ambiente propicio**: Elimina distra

In [16]:
query = "¿Qué consejos tienes para mantener la motivación durante un maratón de estudio o trabajo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro que sí! Mantener la motivación durante un maratón de estudio o trabajo puede ser un verdadero desafío, pero aquí van algunos consejos que pueden ayudarte a mantener el enfoque y la energía:

1. **Establece Metas Claras**: Define qué quieres lograr en cada sesión de estudio o trabajo. Tener un objetivo específico te dará una dirección clara y una razón para seguir adelante.

2. **Divide el Trabajo**: No intentes abarcarlo todo de una vez. Divide tu trabajo en tareas más pequeñas y manejables. Esto hará que el proceso sea menos abrumador y te permitirá celebrar pequeños logros a lo largo del camino.

3. **Crea un Entorno Positivo**: Asegúrate de que tu espacio de trabajo sea cómodo, bien iluminado y libre de distracciones. Un ambiente agradable puede ayudarte a concentrarte mejor.

4. **Incorpora Ejercicio**: Como se menciona en el contexto sobre la relación entre el ejercicio y la función cerebral, hacer pausas para moverte o hacer ejercicio puede aumentar tu energía y mejorar tu

In [17]:
query = "¿Cómo puedo motivarme para salir de mi zona de confort?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Salir de tu zona de confort puede ser un desafío, pero es un paso vital hacia el crecimiento personal y profesional. Aquí hay algunas estrategias que puedes considerar para motivarte:

1. **Visualiza tu objetivo**: Imagina cómo se verá tu vida una vez que hayas salido de esa zona de confort. ¿Qué nuevas oportunidades se abrirán para ti? Crear una imagen clara de tus metas puede impulsarte a dar el primer paso.

2. **Establece pequeños objetivos**: No necesitas hacer un gran salto de inmediato. Comienza con pasos pequeños y manejables. Cada pequeño éxito te dará más confianza y te motivará a seguir adelante.

3. **Rodéate de apoyo**: Comparte tus intenciones con amigos o familiares que te apoyen. Su aliento y motivación pueden ser fundamentales para ayudarte a avanzar.

4. **Acepta el miedo y la incomodidad**: Entiende que sentir miedo o incomodidad es una parte natural del proceso. En lugar de evitar esos sentimientos, reconócelos y usa esa energía para motivarte a seguir adelante.

5.

In [18]:
query = "¿Qué me recomiendas hacer cuando me siento desanimado por los comentarios negativos de otros?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando te sientas desanimado por los comentarios negativos de otros, hay varias estrategias que puedes seguir para elevar tu ánimo y mantenerte motivado. Primero, es fundamental recordar que las críticas pueden ser reflejo de las inseguridades de quienes las hacen, y no determinan tu valor ni tu capacidad.

1. **Cultiva una mentalidad de crecimiento:** Como mencionamos en el contexto de la investigación sobre la motivación, adoptar una "mentalidad de crecimiento" puede ayudarte a ver los comentarios negativos como oportunidades para aprender y mejorar. En lugar de tomarlo como un ataque personal, pregúntate qué puedes aprender de la situación.

2. **Conéctate con tus relaciones positivas:** La investigación ha demostrado que tener buenas relaciones es crucial para nuestra felicidad y bienestar. Rodéate de personas que te apoyen y te inspiren. Comparte tus sentimientos con ellos y permite que te recuerden tus fortalezas y logros.

3. **Reemplaza el tiempo de pantalla por tiempo con pers

In [19]:
query = "¿Cómo puedo encontrar un propósito claro en mi vida?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Encontrar un propósito claro en la vida puede ser un viaje personal y profundo, y hay algunas estrategias que podrías considerar para ayudarte en este proceso. Aquí hay algunas ideas inspiradas en los contextos proporcionados:

1. **Explora tus intereses**: Pregúntate qué actividades te apasionan y te hacen sentir vivo. La libertad de explorar tus propios intereses, como mencionó alguien en los contextos, puede ser fundamental para descubrir lo que realmente te motiva.

2. **Reflexiona sobre tus experiencias**: Al igual que la persona que compartió su historia, a veces nuestras vivencias, incluso las más difíciles, pueden enseñarnos valiosas lecciones. Tómate un tiempo para pensar en cómo tus experiencias pasadas han moldeado tus valores y qué deseas llevar contigo hacia el futuro.

3. **Define tus valores**: Identificar lo que es realmente importante para ti puede ayudarte a establecer un camino claro hacia tu propósito. Considera lo que valoras en la vida, ya sea la libertad, la verd

In [20]:
query = "¿Qué puedo hacer para encontrar motivación cuando no tengo apoyo de los demás?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando te sientes sin apoyo de los demás, es importante recordar que la motivación puede venir de dentro de ti mismo, y hay varias estrategias que puedes utilizar para reavivarla. Aquí hay algunas ideas que podrían ayudarte:

1. **Conéctate contigo mismo**: A veces, es útil mirar hacia adentro y entender qué es lo que realmente te motiva. Tomarte un momento para reflexionar sobre tus propios valores y metas puede ser una gran fuente de inspiración.

2. **Visualiza tu éxito**: Imagina cómo sería alcanzar tus objetivos. Puedes dibujar un mapa de tu sueño, como se mencionó, para tener una representación visual de lo que deseas lograr. Esto puede ayudarte a mantenerte enfocado y motivado.

3. **Establece metas pequeñas y alcanzables**: En lugar de fijarte un objetivo grande que puede parecer abrumador, divídelo en pasos más pequeños y manejables. Cada pequeño logro puede ofrecerte un impulso de motivación.

4. **Desarrolla una mentalidad de crecimiento**: Acepta que el aprendizaje y el cre

In [21]:
query = "¿Cómo puedo desarrollar una mentalidad de crecimiento y resiliencia?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Desarrollar una mentalidad de crecimiento y resiliencia es un viaje emocionante y transformador! Aquí tienes algunas estrategias inspiradas en las charlas que has leído:

1. **Acepta el desafío**: La mentalidad de crecimiento se basa en la creencia de que puedes mejorar con esfuerzo y dedicación. Cuando enfrentes un desafío, en lugar de evitarlo, míralo como una oportunidad para aprender y crecer. Recuerda que la práctica y el esfuerzo son clave.

2. **Aprende de los fracasos**: No temas al fracaso; es una parte esencial del proceso de aprendizaje. Como se menciona en las charlas, la experiencia de las dificultades puede enseñarte lecciones valiosas. Reflexiona sobre lo que puedes aprender de cada tropiezo y úsalo como un peldaño hacia el éxito.

3. **Fomenta conexiones positivas**: Rodéate de personas que te inspiren y te motiven. Las relaciones sólidas son fundamentales para el bienestar emocional. Busca amigos, mentores o colegas que compartan tus valores y te alienten a seguir ade

In [22]:
query = "¿Qué me sugieres para motivarme a mantener hábitos saludables?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Mantener hábitos saludables puede ser un desafío, pero aquí tienes algunas sugerencias inspiradas en los contextos que compartiste:

1. **Visualiza tu éxito**: Como se mencionó en la charla sobre la importancia de crear una imagen de tus metas, dibujar o visualizar tus objetivos de salud puede ser muy poderoso. Imagina cómo te sentirás y cómo lucirás al mantener esos hábitos. Esto puede ayudarte a mantenerte motivado.

2. **Establece metas claras**: Define qué hábitos saludables quieres adoptar y establece objetivos específicos y alcanzables. Ya sea hacer ejercicio regularmente, comer más frutas y verduras, o dormir mejor, tener un plan puede ayudarte a mantenerte enfocado.

3. **Encuentra tu comunidad**: La investigación sugiere que las conexiones sociales son cruciales para la felicidad y el bienestar. Rodéate de personas que también valoran un estilo de vida saludable. Esto puede ser tan simple como unirte a un grupo de ejercicio o compartir tus metas con amigos que te apoye

In [23]:
query = "¿Cómo puedo seguir motivado después de un fracaso o un revés?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando enfrentas un fracaso o un revés, es esencial recordar que estos momentos son parte del viaje hacia el éxito y el crecimiento personal. Aquí hay algunas estrategias que podrías considerar para mantenerte motivado:

1. **Acepta tus emociones**: Es natural sentirse desanimado después de un fracaso. Permítete sentir esas emociones sin juzgarte. Reconocer la tristeza o la frustración es el primer paso para superarlas.

2. **Reenfoca tu perspectiva**: En lugar de ver el fracaso como un final, míralo como una oportunidad para aprender. Como se menciona en la idea del "mindset de crecimiento", la habilidad de aprender no es fija y puede mejorar con el esfuerzo. Reflexiona sobre lo que salió mal y cómo podrías hacerlo diferente la próxima vez.

3. **Construye relaciones de apoyo**: Rodéate de personas que te animen y te apoyen. Las relaciones positivas son fundamentales para nuestra salud y bienestar. Hablar con amigos o familiares sobre tus experiencias puede aliviar la carga emocional 

In [24]:
query = "¿Qué pasos puedo tomar para construir una rutina diaria que me mantenga inspirado?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Para construir una rutina diaria que te mantenga inspirado, hay varias estrategias que podrías considerar:

1. **Visualiza tus metas**: Como se menciona en el contexto, crear una imagen de tu estado actual y del futuro que deseas alcanzar puede ser muy poderoso. Dibuja o escribe sobre tus objetivos y sueños. Esto te dará un mapa claro de hacia dónde te diriges y te motivará a avanzar.

2. **Integra ejercicio en tu rutina**: La investigación sugiere que el ejercicio aeróbico no solo mejora la memoria y la cognición, sino que también puede potenciar tu creatividad. Así que, incluye sesiones de ejercicio en tu día a día, ya sea en el gimnasio, en casa o al aire libre. Esto no solo mantendrá tu cuerpo en forma, sino que también liberará endorfinas que te harán sentir bien y te inspirarán.

3. **Establece un tiempo para la reflexión**: Dedica un momento cada día para reflexionar sobre tus logros y lo que has aprendido. Esto te ayudará a mantenerte enfocado y a reconocer tu progreso, lo cual

In [25]:
query = "¿Cómo puedo motivarme para enfrentar un desafío personal difícil?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Para motivarte a enfrentar un desafío personal difícil, es importante que comiences por visualizar el resultado que deseas lograr. Al igual que se menciona en las charlas sobre la importancia de tener una imagen clara de tus metas, dibujar o escribir sobre tu situación actual y el futuro que deseas puede servir como un poderoso mapa para guiarte. Al hacerlo, no solo clarificas tu objetivo, sino que también activas tu cerebro para que empiece a buscar soluciones.

Además, recuerda que los desafíos son oportunidades para el crecimiento personal. Así como se ha demostrado que el ejercicio aeróbico puede mejorar la memoria y la creatividad, también puedes usar la idea de un "mindset de crecimiento". Esto significa que debes creer que tu capacidad para enfrentar y superar dificultades puede desarrollarse con esfuerzo y dedicación. Cada paso que tomes hacia tu objetivo, por pequeño que sea, es un avance significativo.

No olvides que es normal sentirse ansioso o dudar de ti mismo ante un ret